In [1]:
# @title
from google.colab import userdata
userdata.get('genai_course')
openai_api_key = userdata.get('genai_course')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd /content/drive/MyDrive/GenAI/RAG/RAG with OpenAI

/content/drive/MyDrive/GenAI/RAG/RAG with OpenAI


Perform OCR and transform to images



In [4]:
!pip install pdf2image
!apt-get install -y poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 41 not upgraded.
Need to get 186 kB of archives.
After this operation, 697 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.12 [186 kB]
Fetched 186 kB in 1s (170 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 121713 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.12_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.12) ...
Setting up poppler-utils (22.02.0-2ubuntu0.12) ...
Processing triggers for man-db (2.10.2-1) ...


In [5]:
#Create a function to convert pdfs into images and stores the paths
def pdf_to_images(pdf_path, output_folder):
  image_paths = [] # Initialize image_paths here
  if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    #Convert PDF into images
    images = convert_from_path(pdf_path)
    #Save images and paths
    for i, image in enumerate(images):
      image_path = os.path.join(output_folder, f"page_{i+1}.jpg")
      image.save(image_path, "JPEG")
      image_paths.append(image_path)
  else: # If the output folder already exists, get the list of image paths from it
    for filename in os.listdir(output_folder):
      if filename.endswith(".jpg"):
        image_paths.append(os.path.join(output_folder, filename))

  return image_paths

In [6]:
#Import the Libraries
from pdf2image import convert_from_path
import os

In [7]:
#At the end, we want to have
pdf_path = "Things mother used to make.pdf"
output_folder = "images"
image_paths = pdf_to_images(pdf_path, output_folder)

In [8]:
!pip install OpenAI

In [9]:
!pip install openai

In [10]:
#Import libraries
from openai import OpenAI
import base64 ### package needed to incode images


In [11]:
#Setup connection to openai API
client = OpenAI(api_key=openai_api_key)
model = "gpt-4o-mini"

In [12]:
#Read and incode one image
image_path = "images/page_23.jpg"
with open(image_path, "rb") as image_file:
  image_data = base64.b64encode(image_file.read()).decode('utf-8')
image_data

'...'

In [13]:
#Define the System Prompt
system_prompt = """Please analyze the content of this image and extract any related recipe information."""

In [14]:
#Call the OpenAI API using the chat completion method
response = client.chat.completions.create(
    model=model,
    messages=[
        #Provide the system prompt
          {"role": "system", "content": system_prompt},
        #The user messsage contains the text and image url/path
        {"role": "user", "content": [
            "This is the image from the recipe page.",
            {"type": "image_url",
             "image_url": {"url": f"data:image/jpeg;base64,{image_data}",
             "detail": "low"}}
        ]}
    ]
)

In [15]:
#Display the content
gpt_response=response.choices[0].message.content


In [16]:
from IPython.display import display, Markdown
display(Markdown(gpt_response))

The image contains two recipes for traditional breads:

### Bannocks
**Ingredients:**
- 1 Cupful of Thick Sour Milk
- ½ Cupful of Sugar
- 2 Cupfuls of Flour
- ½ Cupful of Indian Meal
- 1 Teaspoonful of Soda
- A pinch of Salt

**Instructions:**
1. Mix the ingredients until sufficient to drop from a spoon.
2. Drop mixture, the size of a walnut, into boiling fat.
3. Serve warm with maple syrup.

---

### Boston Brown Bread
**Ingredients:**
- 1 Cupful of Rye Meal
- 1 Cupful of Sour Milk
- 1 Cupful of Graham Meal
- 1 Cupful of Molasses
- 1 Cupful of Flour
- ½ Teaspoonful of Salt
- 1 Heaping Teaspoonful of Soda
- ½ Cupful of Sweet Milk

**Instructions:**
1. Mix the meals and salt together.
2. Beat the soda into the molasses until it foams; add sour milk, mix well, and pour into a tin pan which has been well greased. 
3. If desired, you can use a brown bread steamer.

Feel free to ask if you need more details or modifications!

In [17]:
#Define a function to the get the GPT Response and display in Markdown
def get_gpt_response():
  gpt_response = response.choices[0].message.content
  return display(Markdown(gpt_response))
get_gpt_response()

The image contains two recipes for traditional breads:

### Bannocks
**Ingredients:**
- 1 Cupful of Thick Sour Milk
- ½ Cupful of Sugar
- 2 Cupfuls of Flour
- ½ Cupful of Indian Meal
- 1 Teaspoonful of Soda
- A pinch of Salt

**Instructions:**
1. Mix the ingredients until sufficient to drop from a spoon.
2. Drop mixture, the size of a walnut, into boiling fat.
3. Serve warm with maple syrup.

---

### Boston Brown Bread
**Ingredients:**
- 1 Cupful of Rye Meal
- 1 Cupful of Sour Milk
- 1 Cupful of Graham Meal
- 1 Cupful of Molasses
- 1 Cupful of Flour
- ½ Teaspoonful of Salt
- 1 Heaping Teaspoonful of Soda
- ½ Cupful of Sweet Milk

**Instructions:**
1. Mix the meals and salt together.
2. Beat the soda into the molasses until it foams; add sour milk, mix well, and pour into a tin pan which has been well greased. 
3. If desired, you can use a brown bread steamer.

Feel free to ask if you need more details or modifications!

In [18]:
#Define improved system prompt
system_prompt = """
Please analyze the content of this image and extract any recipe related information into structured components.
Specifically, extract the recipe title, ingredients, cuisine type, dish type, any relevant tags and step-by-step directions.
The oputput should be formatted in a way suited for embedding in Retreival Augmented Generation system.
"""

In [19]:
#Call the API to extract the information
response = client.chat.completions.create(
    model=model,
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content":[
            "This is the image from the recipe page.",
              {"type": "image_url",
              "image_url": {"url": f"data:image/jpeg;base64,{image_data}",
                          "detail": "low"}}
        ]}
    ],
    temperature=0
)

In [20]:
extracted_recipes = []
#Reading and decoding images
for image_path in image_paths:
  print(f"Processing {image_path}")
  with open(image_path, "rb") as image_file:
    image_data = base64.b64encode(image_file.read()).decode('utf-8')

  #Call the OpenAI API
  response = client.chat.completions.create(
    model=model,
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content":[
            "This is the image from the recipe page.",
              {"type": "image_url",
              "image_url": {"url": f"data:image/jpeg;base64,{image_data}",
                          "detail": "low"}}
        ]}
    ],
    temperature=0
)
  # Extract The Content and store
  gpt_response = response.choices[0].message.content
  extracted_recipes.append({"image_path": image_path, "recipe_info": gpt_response})
  print(f"Extracted information from {image_path} :\n{gpt_response}\n")

Streaming output truncated to the last 5000 lines.

---

### Additional Recipe:
**New England Buns**
- 1 Cupful of Milk
- 1 and 1/3 Cups of Sugar
- 2/3 Cupful of Butter or Lard
- 1 Teaspoonful of Yeast
- 1/2 Teaspoonful of Soda
- 1/2 Teaspoonful of Salt
- 1 Teaspoonful of Extract of Lemon
- Flour enough for Dough

### Directions for New England Buns:
1. Dissolve the yeast in a half-cup of cold water.
2. Scale the milk and water, and add the sugar and flour to make a thin batter.
3. Let it rise to twice its bulk.
4. When light and frothy, add the rest of the ingredients, mixing well.
5. Cut out with a cookie-cutter, and let rise again.
6. Bake in a moderate oven.

Processing images/page_31.jpg
Extracted information from images/page_31.jpg :
Here’s the structured information extracted from the recipe:

### Recipe Title:
Nut Bread and Oatmeal Bread

### Ingredients:
**Nut Bread:**
- 2½ Cups of Flour
- 3 Teaspoons of Baking Powder
- ¾ Cup of Milk
- ½ Cup of Sugar
- 1 Cup of Chopped Nuts (o

Let's start by running the cell that generates the `image_paths`. Please run the following cell:

In [21]:
#Print the info from the page with the improved prompt
get_gpt_response()

I'm unable to analyze the content of the image directly. If you can provide the text or details from the recipe, I can help you structure that information!

In [22]:
extracted_recipes

[{'image_path': 'images/page_1.jpg',
  'recipe_info': "I can't extract specific recipe information from the image you provided. If you have text or details from the recipe, feel free to share, and I can help you organize that information!"},
 {'image_path': 'images/page_2.jpg',
  'recipe_info': "I can't extract any recipe-related information from the image you provided, as it does not contain any recipe content. If you have another image with a recipe, feel free to share it!"},
 {'image_path': 'images/page_3.jpg',
  'recipe_info': 'It seems that the image you provided does not contain any recipe-related information. It appears to be a page from a library catalog or a book, rather than a recipe. If you have another image or specific text related to a recipe, please share that, and I can help extract the relevant information.'},
 {'image_path': 'images/page_4.jpg',
  'recipe_info': "I can't extract any recipe-related information from the image you provided, as it does not contain any vis

In [23]:
#Filter out non-recipe content based on key-recipe related terms
filtered_recipes = []
for recipe in extracted_recipes:
  if any(keyword in recipe["recipe_info"].lower() for keyword in ["ingredients", "instructions", "recipe title"]):
    filtered_recipes.append(recipe)
  else:
    print(f"Skipping {recipe['image_path']} due to non-recipe content")

Skipping images/page_1.jpg due to non-recipe content
Skipping images/page_2.jpg due to non-recipe content
Skipping images/page_3.jpg due to non-recipe content
Skipping images/page_4.jpg due to non-recipe content
Skipping images/page_6.jpg due to non-recipe content
Skipping images/page_8.jpg due to non-recipe content
Skipping images/page_9.jpg due to non-recipe content
Skipping images/page_10.jpg due to non-recipe content
Skipping images/page_11.jpg due to non-recipe content
Skipping images/page_12.jpg due to non-recipe content
Skipping images/page_17.jpg due to non-recipe content
Skipping images/page_19.jpg due to non-recipe content
Skipping images/page_20.jpg due to non-recipe content
Skipping images/page_21.jpg due to non-recipe content
Skipping images/page_22.jpg due to non-recipe content
Skipping images/page_106.jpg due to non-recipe content
Skipping images/page_107.jpg due to non-recipe content
Skipping images/page_108.jpg due to non-recipe content
Skipping images/page_109.jpg due

In [24]:
import json


In [25]:
#Define the output file path
output_file = "recipe_info.json"

# Write the filtered list to a JSON file
with open(output_file, "w") as json_file:
  json.dump(filtered_recipes, json_file, indent = 4)


Embeddings

In [26]:
#import libraries
import numpy as np

In [27]:
#Load The filtered recipes
with open("recipe_info.json", "r") as json_file:
  filtered_recipes = json.load(json_file)

Another option would be to organize per recipe but should be done in the preprocessing step

In [28]:
#Generate embeeding for each recipe
recipe_texts = [recipe["recipe_info"] for recipe in filtered_recipes]
embedding_response = client.embeddings.create(
    input=recipe_texts,
    model="text-embedding-3-large"
)

In [29]:
#Extract the embeddings
embeddings = [data.embedding for data in embedding_response.data]

In [30]:
#Convert The Embedding
embedding_matrix = np.array(embeddings)
embedding_matrix

array([[-0.01444156, -0.01923144, -0.02207948, ..., -0.00030881,
        -0.03115579,  0.00650877],
       [-0.02127117, -0.03587804, -0.01842735, ..., -0.00195871,
        -0.02632684,  0.00643808],
       [ 0.0139512 , -0.02455585, -0.01172017, ...,  0.0029065 ,
        -0.02309264,  0.01900725],
       ...,
       [-0.02258907, -0.01968518, -0.01494264, ...,  0.0078872 ,
        -0.03394197,  0.01504479],
       [ 0.00138103, -0.01919352, -0.01215501, ..., -0.00074975,
         0.00325581, -0.00528882],
       [-0.02259134, -0.03264299, -0.0115658 , ..., -0.0056585 ,
        -0.01081228,  0.00443936]])

In [31]:
#Verify The Embedding Matrix
print(f"Generated {len(embeddings)} embeddings of size {len(embeddings[0])} for {len(filtered_recipes)} recipes.")


Generated 112 embeddings of size 3072 for 112 recipes.


Retrieval System

In [32]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 86.5 MB/s eta 0:00:00


In [33]:
import faiss

In [34]:
#Print the embedding matrix shape
print(f"Embedding matrix shape: {embedding_matrix.shape}")

Embedding matrix shape: (112, 3072)


In [35]:
#initialize the FAISS index
index = faiss.IndexFlatL2(embedding_matrix.shape[1])
index.add(embedding_matrix)

In [36]:
#Save the index
faiss.write_index(index, "filtered_recipe_index.index")

In [37]:
#Save the Metadata |||| Allows you to assess outcomes to determine if results are reasonable
metadata = [{"recipe_info": recipe ["recipe_info"],
             "image_path": recipe["image_path"]} for recipe in filtered_recipes]
with open("filtered_recipe_metadata.json", "w") as json_file:
  json.dump(metadata, json_file, indent= 4)

In [48]:
#Define a function to query the embedddings
def query_embeddings(query, index, metadata, k=5):
  """
  Queries the FAISS index with a given query and returns the top k relevant recipes.

  Args:
    query (str): The query string.
    index (faiss.Index): The FAISS index.
    metadata (list): A list of dictionaries containing recipe metadata.
    k (int): The number of top results to retrieve.

  Returns:
    tuple: A tuple containing:
      - distances (numpy.ndarray): The distances of the retrieved items from the query.
      - results (list): A list of dictionaries, where each dictionary is the metadata
                       of a relevant recipe.
  """
  # Generate the embeddings for the query
  query_embedding = client.embeddings.create(
      input=[query],
      model="text-embedding-3-large"
  ).data[0].embedding
  # Convert the query embedding to a numpy array
  query_vector = np.array(query_embedding).reshape(1, -1)
  # Search the FAISS index
  distances, indices = index.search(query_vector, min(k, len(metadata)))

  # Retrieve the relevant metadata based on the indices
  results = [metadata[i] for i in indices[0]]

  return distances, results

In [49]:
query = "cakes"
distances, results = query_embeddings(query, index, metadata, k=3)

print(f"Query: {query}\n")
print("Top 3 relevant recipes:")
for i, (dist, recipe) in enumerate(zip(distances[0], results)):
    print(f"\nRank {i+1} (Distance: {dist:.4f}):")
    print(f"Image Path: {recipe['image_path']}")
    print(f"Recipe Info: {recipe['recipe_info'][:200]}...") # Displaying first 200 characters of recipe info

Query: cakes

Top 3 relevant recipes:

Rank 1 (Distance: 1.3552):
Image Path: images/page_40.jpg
Recipe Info: Here’s the structured information extracted from the recipe:

### Recipe Title:
One, Two, Three, Four Cake

### Ingredients:
- 1 Cupful of Butter
- 2 Cupfuls of Sugar
- 2 Cupfuls of Flour
- 2 Teaspoon...

Rank 2 (Distance: 1.3709):
Image Path: images/page_36.jpg
Recipe Info: Here’s the structured information extracted from the recipe page:

### Recipe Title
Cream Cake

### Ingredients
- 2 Cups of Flour
- 1 Cup of Cream of Tartar
- 1 Cup of Sugar
- 1 Cup of Butter
- 2 Cups...

Rank 3 (Distance: 1.3986):
Image Path: images/page_34.jpg
Recipe Info: Here’s the structured information extracted from the recipe:

### Recipe Title:
Spider Cake

### Ingredients:
- 2 Cups of Bread Flour
- 1 Teaspoon of Soda
- 1/3 Cup of Lard
- 2 Teaspoons of Cream of T...
